In [117]:
'''
https://preppindata.blogspot.com/2023/02/2023-week-8-taking-stock.html
'''
import pandas as pd

In [118]:
dfs = []
for i in range(1,13):
    if i == 1:
        dfs.append(pd.read_csv('MOCK_DATA.csv'))
    else:
        s = f'MOCK_DATA-{i}.csv'
        dfs.append(pd.read_csv(s))

print(len(dfs))

12


In [119]:
dfs[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1000 non-null   int64 
 1   first_name      1000 non-null   object
 2   last_name       1000 non-null   object
 3   Ticker          1000 non-null   object
 4   Sector          791 non-null    object
 5   Market          1000 non-null   object
 6   Stock Name      1000 non-null   object
 7   Market Cap      853 non-null    object
 8   Purchase Price  1000 non-null   object
dtypes: int64(1), object(8)
memory usage: 70.4+ KB


In [120]:
dfs[1].head(5)

,id,first_name,last_name,Ticker,Sector,Market,Stock Name,Market Cap,Purchase Price
0,1,Sybyl,Huxley,TMUS,Public Utilities,NASDAQ,"T-Mobile US, Inc.",$52.45B,$38271.32
1,2,Perle,Scarfe,RELL,Consumer Non-Durables,NASDAQ,"Richardson Electronics, Ltd.",$75.94M,$1431.02
2,3,Montgomery,Paliser,RBS,Finance,NYSE,Royal Bank Scotland plc (The),$37.92B,$84657.21
3,4,Kirstyn,Falco,TEGP,Public Utilities,NYSE,"Tallgrass Energy GP, LP",$3.69B,$5030.66
4,5,Eustacia,Rea,WILC,Consumer Non-Durables,NASDAQ,"G. Willi-Food International, Ltd.",$92.42M,$21845.51


### Create a 'file date' using the month found in the file name

In [121]:
for i in range(len(dfs)):
    dfs[i]['File Date'] = f'1/{i+1}/2023'

dfs[2].head(5)

,id,first_name,last_name,Ticker,Sector,Market,Stock Name,Market Cap,Purchase Price,File Date
0,1,Gay,Howgate,LUB,Consumer Services,NYSE,"Luby's, Inc.",$88.73M,$44389.17,1/3/2023
1,2,Shara,Kember,NRT,Energy,NYSE,North European Oil Royality Trust,$61.21M,$60710.96,1/3/2023
2,3,Fawn,Hayers,RUBI,Technology,NYSE,"The Rubicon Project, Inc.",$275.01M,$10714.77,1/3/2023
3,4,Colette,Ricciardelli,PHMD,Health Care,NASDAQ,"PhotoMedex, Inc.",$6.29M,$17801.56,1/3/2023
4,5,Collete,Kettlesing,EVAR,Health Care,NASDAQ,"Lombard Medical, Inc.",$13.04M,$29170.58,1/3/2023


### Clean the Market Cap value to ensure it is the true value as 'Market Capitalisation'
        Remove any rows with 'n/a'

In [122]:
for i in range(len(dfs)):
    dfs[i]=dfs[i].dropna(subset=['Market Cap']).reset_index(drop=True)

### Categorise the Purchase Price into groupings
        0 to 24,999.99 as 'Low'
        25,000 to 49,999.99 as 'Medium'
        50,000 to 74,999.99 as 'High'
        75,000 to 100,000 as 'Very High'
### Categorise the Market Cap into groupings
        Below $100M as 'Small'
        Between $100M and below $1B as 'Medium'
        Between $1B and below $100B as 'Large' 
        $100B and above as 'Huge'

In [123]:
dfs[0].loc[10]

id                                         16
first_name                            Jacinta
last_name                            Shorland
Ticker                                   USNA
Sector                      Consumer Durables
Market                                   NYSE
Stock Name        USANA Health Sciences, Inc.
Market Cap                              $1.6B
Purchase Price                      $26252.35
File Date                            1/1/2023
Name: 10, dtype: object

In [124]:
def price2int(x):
    sym = x[-1]
    x = x[1:-1]
    if sym == 'M':
        return int(float(x)*10**6)
    elif sym == 'B':
        return int(float(x)*10**9)
    else: 
        return float(x[1:])


for i in range(len(dfs)):
    dfs[i]['Purchase Price'] = [float(x.replace('$', '')) for x in dfs[i]['Purchase Price']]
    for j in range(len(dfs[i])):
        # Purchase Price Categorisation
        if dfs[i].loc[j ,'Purchase Price'] <= 24999.99:
            dfs[i].loc[j,'Purchase Price Categorisation'] = 'Low'

        elif (dfs[i].loc[j ,'Purchase Price'] >= 25000) and dfs[i].loc[j ,'Purchase Price'] <= 49999.99 :
            dfs[i].loc[j,'Purchase Price Categorisation'] = 'Medium'

        elif (dfs[i].loc[j ,'Purchase Price'] >= 50000) and dfs[i].loc[j ,'Purchase Price'] <= 74999.99 :
            dfs[i].loc[j,'Purchase Price Categorisation'] = 'High'
        
        elif (dfs[i].loc[j ,'Purchase Price'] >= 75000) and dfs[i].loc[j ,'Purchase Price'] <= 100000 :
            dfs[i].loc[j,'Purchase Price Categorisation'] = 'Very High'

        # Market Capitalisation Categorisation
        dfs[i].loc[j,'Market Cap'] = price2int(dfs[i].loc[j,'Market Cap'])
        if dfs[i].loc[j,'Market Cap'] < 100*10**6:
            dfs[i].loc[j,'Market Capitalisation Categorisation'] = 'Small'

        elif (dfs[i].loc[j,'Market Cap'] >= 100*10**6) and (dfs[i].loc[j,'Market Cap'] < 1*10**9):
            dfs[i].loc[j,'Market Capitalisation Categorisation'] = 'Medium'

        elif (dfs[i].loc[j,'Market Cap'] >= 1*10**9) and (dfs[i].loc[j,'Market Cap'] < 100*10**9):
            dfs[i].loc[j,'Market Capitalisation Categorisation'] = 'Large'

        elif (dfs[i].loc[j,'Market Cap'] >= 100*10**9):
            dfs[i].loc[j,'Market Capitalisation Categorisation'] = 'Huge'

In [125]:
dfs[0].head(5)

,id,first_name,last_name,Ticker,Sector,Market,Stock Name,Market Cap,Purchase Price,File Date,Purchase Price Categorisation,Market Capitalisation Categorisation
0,2,Lelia,Gresley,GILD,Health Care,NASDAQ,"Gilead Sciences, Inc.",83790000000,77627.18,1/1/2023,Very High,Large
1,3,Leicester,Roff,NMZ,NaN,NYSE,Nuveen Municipal High Income Opportunity Fund,794980000,40676.73,1/1/2023,Medium,Medium
2,5,Julita,Spradbery,INSI,NaN,NYSE,Insight Select Income Fund,211520000,41908.66,1/1/2023,Medium,Medium
3,6,Steven,McGrale,AP,Capital Goods,NYSE,Ampco-Pittsburgh Corporation,175480000,163.84,1/1/2023,Low,Medium
4,8,Elly,Dono,SCM,NaN,NYSE,Stellus Capital Investment Corporation,213680000,55985.51,1/1/2023,High,Medium


### Rank the highest 5 purchases per combination of: file date, Purchase Price Categorisation and Market Capitalisation Categorisation.

In [126]:
total_df = pd.concat(dfs)
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10109 entries, 0 to 839
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    10109 non-null  int64  
 1   first_name                            10109 non-null  object 
 2   last_name                             10109 non-null  object 
 3   Ticker                                10109 non-null  object 
 4   Sector                                8599 non-null   object 
 5   Market                                10109 non-null  object 
 6   Stock Name                            10109 non-null  object 
 7   Market Cap                            10109 non-null  object 
 8   Purchase Price                        10109 non-null  float64
 9   File Date                             10109 non-null  object 
 10  Purchase Price Categorisation         10109 non-null  object 
 11  Market Capitalisa

In [127]:
total_df['rank'] = total_df.groupby(['File Date', 'Purchase Price Categorisation', 'Market Capitalisation Categorisation'])['Purchase Price'].rank('max')
total_df['rank'] = total_df['rank'].astype(int)
total_df = total_df.reset_index()
total_df.head(10)

,index,id,first_name,last_name,Ticker,Sector,Market,Stock Name,Market Cap,Purchase Price,File Date,Purchase Price Categorisation,Market Capitalisation Categorisation,rank
0,0,2,Lelia,Gresley,GILD,Health Care,NASDAQ,"Gilead Sciences, Inc.",83790000000,77627.18,1/1/2023,Very High,Large,9
1,1,3,Leicester,Roff,NMZ,NaN,NYSE,Nuveen Municipal High Income Opportunity Fund,794980000,40676.73,1/1/2023,Medium,Medium,55
2,2,5,Julita,Spradbery,INSI,NaN,NYSE,Insight Select Income Fund,211520000,41908.66,1/1/2023,Medium,Medium,58
3,3,6,Steven,McGrale,AP,Capital Goods,NYSE,Ampco-Pittsburgh Corporation,175480000,163.84,1/1/2023,Low,Medium,1
4,4,8,Elly,Dono,SCM,NaN,NYSE,Stellus Capital Investment Corporation,213680000,55985.51,1/1/2023,High,Medium,19
5,5,9,Guilbert,Fretter,CHSCM,Consumer Services,NASDAQ,CHS Inc,542670000,79606.74,1/1/2023,Very High,Medium,10
6,6,11,Maurie,Grigoli,JMBA,Consumer Services,NASDAQ,"Jamba, Inc.",107460000,88103.50,1/1/2023,Very High,Medium,33
7,7,12,Othilia,Ortler,FLOW,Technology,NYSE,"SPX FLOW, Inc.",1640000000,9811.38,1/1/2023,Low,Large,27
8,8,14,Erminie,Lis,JHD,NaN,NYSE,Nuveen High Income December 2019 Target Term Fund,277100000,24418.39,1/1/2023,Low,Medium,73
9,9,15,Melva,Fennelow,COO,Health Care,NYSE,"Cooper Companies, Inc. (The)",11670000000,30866.15,1/1/2023,Medium,Large,26


In [129]:
rank_df = pd.DataFrame(columns=total_df.columns)
for i in range(len(total_df)):
    if total_df.loc[i,'rank'] >= 1 and total_df.loc[i,'rank'] <= 5:
        rank_df.loc[len(rank_df)] = total_df.loc[i]

rank_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 831 entries, 0 to 830
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   index                                 831 non-null    object
 1   id                                    831 non-null    object
 2   first_name                            831 non-null    object
 3   last_name                             831 non-null    object
 4   Ticker                                831 non-null    object
 5   Sector                                719 non-null    object
 6   Market                                831 non-null    object
 7   Stock Name                            831 non-null    object
 8   Market Cap                            831 non-null    object
 9   Purchase Price                        831 non-null    object
 10  File Date                             831 non-null    object
 11  Purchase Price Categorisation   

In [ ]:
rank_df.to_csv('output.csv', index=False)